# SQA data joiner

In [ ]:
# Import librarys

import pandas as pd
import numpy as np

# Functions

In [ ]:
# Define functions to read sheets

def read_sheet_new(year, level, header):

    # read in xlsx
    df = pd.read_excel('./data_stats_2022/attainment-statistics-december-' + year +'.xlsx', 
                        sheet_name = level)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_2022/attainment-statistics-december-' + year +'.xlsx', 
                        sheet_name = level, skiprows = header_index + 1,
                          na_values = ['***', '-', ' -', '[c]', '[z]'], thousands = ',')
    
    return df

def read_single_sheet(year, level, header, sheet_to_find):
    
    # create list of sheetnames
    sheet_names = pd.ExcelFile('./data_stats_2000/ASR' + year + '_' + level + '.xls').sheet_names
    
    # format the list of sheet names
    sheet_names = [name.lower() for name in sheet_names]
    
    # get the index that matches our sheet to find
    index = sheet_names.index(sheet_to_find)
    
    # feed this index into pandas
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = index)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = index, skiprows = header_index + 1,
                          na_values = ['***', '-', ' -', '[c]', '[z]'])
    
    return df

def read_sheet(year, level, header, sheets_to_find):
        
    # create list of sheetnames
    tabs = pd.ExcelFile('./data_stats_2000/ASR' + year + '_' + level + '.xls').sheet_names
    # Use key to find correct substring to search sheetnames for
    sheets = sheets_to_find
    # Find correct sheetname, case insensitive search
    name = [s for s in tabs if any(tabs in s.lower() for tabs in sheets)][0]

    # read in xls
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = name)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_2000/ASR' + year + '_' + level + '.xls', 
                        sheet_name = name, skiprows = header_index + 1,
                          na_values = ['***', '-', ' -', '[c]', '[z]'])
    
    return df

def read_old_single_sheet(year, header, sheet_to_find, skip):
    
    # create list of sheetnames
    sheet_names = pd.ExcelFile('./data_stats_1986/Stats' + year + '.xls').sheet_names
    
    # format the list of sheet names
    sheet_names = [name.lower() for name in sheet_names]
    
    # get the index that matches our sheet to find
    index = sheet_names.index(sheet_to_find)

    # feed this index into pandas
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index)
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index, skiprows = header_index + skip,
                          na_values = ['***', '-', ' -', '[c]', '[z]'])
    
    return df

def read_old_higher_single_sheet(year, header, sheet_to_find):
    
    # create list of sheetnames
    sheet_names = pd.ExcelFile('./data_stats_1986/Stats' + year + '.xls').sheet_names
    
    # format the list of sheet names
    sheet_names = [name.lower() for name in sheet_names]
    
    # get the index that matches our sheet to find
    index = sheet_names.index(sheet_to_find)

    # feed this index into pandas
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index)
    
    # Delete first column
    del df[df.columns[0]]
    
    # Look for header index
    header_index = df[df.iloc[:, 0].str.match(header, na = False, case = False)].index[0]

    # Read in sheet skipping the correct number of rows
    df = pd.read_excel('./data_stats_1986/Stats' + year + '.xls', 
                        sheet_name = index, skiprows = header_index + 3,
                          na_values = ['***', '-', ' -', '[c]', '[z]'])
    

    # Delete first column
    del df[df.columns[0]]
    
    return df

In [ ]:
## Define functions to tidy data

def tidy_df(df):
    
    #### Fix usual issues with all strings
    
    # Capitalise headers
    df.columns = df.columns.astype(str).str.upper()
    
    # Rename subject column
    df = df.rename(columns={'TITLE': 'SUBJECT'})
    
    # Capitalise cells
    df = df.applymap(lambda x: x.upper() if type(x) is str else x)

    # Strip whitespace
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)

    # Remove parenthesis
    df = df.applymap(lambda x: x.replace('(', '') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(')', '') if type(x) is str else x)
    
    # Remove linebreaks
    df = df.applymap(lambda x: x.replace('\n', '') if type(x) is str else x)

    # Replace annoying substrings
    df = df.applymap(lambda x: x.replace(' AND ', ' & ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' – ', ' - ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' / ', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('/ ', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' /', '/') if type(x) is str else x)
    
    df = df.applymap(lambda x: x.replace(' - ', ': ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' : ', ': ') if type(x) is str else x)
    
    df = df.applymap(lambda x: x.replace('  ', ' ') if type(x) is str else x)
    
    # Drop rows with minimum number of 2 non-null values
    df = df.dropna(thresh=2)

    # Drop rows with columns number of 2 non-null values
    df = df.dropna(axis='columns', thresh=2)
    
    return df

In [ ]:
# Define functions to count grades

def grade_count_newer(df):
    
    # Change columns from percentages to counts
    df['A'] = df['GRADE A COUNT']
    df['B'] = df['GRADE A TO B COUNT'] - df['GRADE A COUNT']
    df['C'] = df['GRADE A TO C COUNT'] - df['GRADE A TO B COUNT']
    df['D'] = df['GRADE A TO D COUNT'] - df['GRADE A TO C COUNT']
    
    return df

def grade_count_new(df):
    
    # Change columns from percentages to counts
    df['A'] = df['GRADE A COUNT']
    df['B'] = df['GRADE A-B COUNT'] - df['GRADE A COUNT']
    df['C'] = df['GRADE A-C COUNT'] - df['GRADE A-B COUNT']
    df['D'] = df['GRADE A-D COUNT'] - df['GRADE A-C COUNT']
    
    return df

def grade_count(df):
    
    # Change columns from percentages to counts
    df['A'] = df['A'] * df['ENTRIES'] // 100
    df['B'] = df['B'] * df['ENTRIES'] // 100
    df['C'] = df['C'] * df['ENTRIES'] // 100
    df['D'] = df['D'] * df['ENTRIES'] // 100
    
    return df

def pass_fail(df):
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    # Add column with na count
    df['NA'] = df['ENTRIES'] - df['A'] - df['B'] - df['C'] - df['D']
    
    # Add column with pass count
    df['PASS'] = df['ENTRIES'] - df['NA']
    
    return df

def pass_fail_no_d(df):
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    # Add column with na count
    df['NA'] = df['ENTRIES'] - df['A'] - df['B'] - df['C']
    
    # Add column with pass count
    df['PASS'] = df['ENTRIES'] - df['NA']
    
    return df

def pass_fail_ordinary(df):
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    # Add column with na count
    df['NA'] = df['ENTRIES'] - df['1'] - df['2'] - df['3'] - df['4'] - df['5']
    
    # Add column with pass count
    df['PASS'] = df['ENTRIES'] - df['NA']
    
    return df

In [ ]:
# Define functions to select columns

def select_old_columns(df):
    
    # Rename first column to subject
    df = df.rename(columns = {df.columns[0]:'SUBJECT'})
    
    # Select total entries and grades columns
    ls = ['SUBJECT', 'ENTRIES.2', '1.2', '2.2', '3.2', '4.2', '5.2', '6.2', '7.2']
    df = df[ls]
    
    # Remove '.2' suffix
    df.columns = df.columns.str.removesuffix('.2')
    
    return df

def select_old_higher_columns(df):
    
    # Rename first column to subject
    df = df.rename(columns = {df.columns[0]:'SUBJECT'})
    
    # Select total entries and grades columns
    ls = ['SUBJECT', 'ENTRIES.2', 'A.2', 'B.2', 'C.2', 'D.2']
    df = df[ls]
    
    # Remove '.2' suffix
    df.columns = df.columns.str.removesuffix('.2')
    
    # Convert strings in dodgy rows in entries column to NaN
    df['ENTRIES'] = pd.to_numeric(df['ENTRIES'], errors = 'coerce')
    # Drop rows with NaN in Entries column
    df = df[df['ENTRIES'].notna()]
    
    return df

def select_old_ordinary_columns(df):
    
        # Rename first column to subject
    df = df.rename(columns = {df.columns[0]:'SUBJECT'})
    
    # Select total entries and grades columns
    ls = ['SUBJECT', 'ENTRIES.2', '1.2', '2.2', '3.2', '4.2', '5.2']
    df = df[ls]
    
    # Remove '.2' suffix
    df.columns = df.columns.str.removesuffix('.2')
    
    return df

# Read in and process excel files

In [ ]:
# Create empty list for dfs
frames = []

In [ ]:
# Define level and years
levels = ['National_4']
years = ['2023']

# Define header to search for
header = 'subject'

# Create empty dataframe
dfl = pd.DataFrame()

# Loop through levels
for level in levels:
    
    # Loop through years
    for year in years:
        
        # Read and tidy data
        df = read_sheet_new(year, level, header)
        df = tidy_df(df)
        
        # Create a df for passes
        df1 = df[['SUBJECT','AWARDED COUNT 2023']].copy()
        # Rename headers
        df1.columns = ['SUBJECT', 'COUNT']
        # Add in year and level columns
        df1['YEAR'] = int(year)
        df1['LEVEL'] = level
        df1['GRADE'] = 'PASS'
        # Append to df list
        frames.append(df1)
        
        # Create a df for entries
        df2 = df[['SUBJECT','ENTRIES 2023']].copy()
        # Rename headers
        df2.columns = ['SUBJECT', 'COUNT']
        # Add in year and level columns
        df2['YEAR'] = int(year)
        df2['LEVEL'] = level
        df2['GRADE'] = 'ENTRIES'
        # Append to df list
        frames.append(df2)
        
        # Create a df for fails
        df3 = df[['SUBJECT','ENTRIES 2023', 'AWARDED COUNT 2023']].copy()
        # Calculate number of no awards
        df3['COUNT'] = df3['ENTRIES 2023'] - df3['AWARDED COUNT 2023']
        # Drop headers used for calculation
        df3 = df3.drop(['ENTRIES 2023', 'AWARDED COUNT 2023'], axis=1)
        # Add in year and level columns
        df3['YEAR'] = int(year)
        df3['LEVEL'] = level
        df3['GRADE'] = 'NA'
        # Append to df list
        frames.append(df3)

In [ ]:
# Define level and years
levels = ['National_4']
years = ['2022']

# Define header to search for
header = 'subject'

# Create empty dataframe
dfl = pd.DataFrame()

# Loop through levels
for level in levels:
    
    # Loop through years
    for year in years:
        
        # Read and tidy data
        df = read_sheet_new(year, level, header)
        df = tidy_df(df)
        
        # Create a df for passes
        df1 = df[['SUBJECT','AWARDED COUNT 2022']].copy()
        # Rename headers
        df1.columns = ['SUBJECT', 'COUNT']
        # Add in year and level columns
        df1['YEAR'] = int(year)
        df1['LEVEL'] = level
        df1['GRADE'] = 'PASS'
        # Append to df list
        frames.append(df1)
        
        # Create a df for entries
        df2 = df[['SUBJECT','ENTRIES 2022']].copy()
        # Rename headers
        df2.columns = ['SUBJECT', 'COUNT']
        # Add in year and level columns
        df2['YEAR'] = int(year)
        df2['LEVEL'] = level
        df2['GRADE'] = 'ENTRIES'
        # Append to df list
        frames.append(df2)
        
        # Create a df for fails
        df3 = df[['SUBJECT','ENTRIES 2022', 'AWARDED COUNT 2022']].copy()
        # Calculate number of no awards
        df3['COUNT'] = df3['ENTRIES 2022'] - df3['AWARDED COUNT 2022']
        # Drop headers used for calculation
        df3 = df3.drop(['ENTRIES 2022', 'AWARDED COUNT 2022'], axis=1)
        # Add in year and level columns
        df3['YEAR'] = int(year)
        df3['LEVEL'] = level
        df3['GRADE'] = 'NA'
        # Append to df list
        frames.append(df3)

In [ ]:
# Define level and years
levels = ['National_5', 'Higher', 'Advanced_Higher']
years = ['2023']

# Define header to search for
header = 'subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet_new(year, level, header)
        df = tidy_df(df)
        
        # Select important columns
        df = df.filter(regex='SUBJECT|2023')
        
        # Remove ' ' + year suffix
        df.columns = df.columns.str.removesuffix(' ' + year)
        
        # Calculate grade counts from percentages
        df = grade_count_newer(df)

        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['National_5', 'Higher', 'Advanced_Higher']
years = ['2022']

# Define header to search for
header = 'subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet_new(year, level, header)
        df = tidy_df(df)
        
        # Select important columns
        df = df.filter(regex='SUBJECT|2022')
        
        # Remove ' ' + year suffix
        df.columns = df.columns.str.removesuffix(' ' + year)
        
        # Calculate grade counts from percentages
        df = grade_count_new(df)

        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['National_5', 'Higher', 'Advanced_Higher']

years = ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']

# Define sheetnames to grab and header and columns to search for
sheets = ['table 4','4b']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()


for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Calculate grade counts from percentages
        df = grade_count(df)

        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
level = 'New_Higher'
year = '2015'

sheets = ['table 4']
header = 'title'

# Create empty dataframe
dfl = pd.DataFrame()

# Read and tidy data
df = read_sheet(year, level, header, sheets)
df = tidy_df(df)

# Calculate grade counts from percentages
df = grade_count(df)

# Calculate pass and fails
df = pass_fail(df)

# Change into long format
dfl = pd.melt(df, id_vars =['SUBJECT'], 
        value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                var_name ='GRADE', value_name ='COUNT')

# Add in year and level columns
dfl['YEAR'] = int(year)
dfl['LEVEL'] = level

# Append to df list
frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Intermediate2', 'Intermediate1']

years = ['2015', '2014']

# Define sheetnames to grab and header and columns to search for
sheets = ['table 4','4b']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Calculate grade counts from percentages
        df = grade_count(df)

        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Intermediate2', 'Intermediate1', 'Higher', 'Advanced_Higher']
years = ['2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        ### Look at graded courses
        if 'PASSES IN UNGRADED COURSES' in df.columns.values:
            df1 = df[pd.isnull(df['PASSES IN UNGRADED COURSES'])].copy()
        else:
            df1 = df
        
        # Calculate pass and fails
        df1 = pass_fail(df1)

        # Change into long format
        dfl = pd.melt(df1, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level

        # Append to df list
        frames.append(dfl)
        
        
        ### Look at ungraded courses
        if 'PASSES IN UNGRADED COURSES' in df.columns.values:
            
            # Drop rows with grades
            df2 = df[pd.notnull(df['PASSES IN UNGRADED COURSES'])].copy()
            
            # Sort column name
            df2 = df2.rename(columns = {'PASSES IN UNGRADED COURSES': 'PASS'})
        
            # Add column with na count
            df2['NA'] = df['ENTRIES'] - df2['PASS']

            # Change into long format
            dfl = pd.melt(df2, id_vars =['SUBJECT'], 
                    value_vars = ['ENTRIES', 'PASS', 'NA'],
                            var_name ='GRADE', value_name ='COUNT')

            # Add in year and level columns
            dfl['YEAR'] = int(year)
            dfl['LEVEL'] = level
            
            # Append to df list
            frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Advanced_Higher']
years = ['2003', '2002', '2001']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Rename comp column
        df = df.rename(columns={'COMP': 'D'})
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Intermediate1']
years = ['2003', '2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Calculate pass and fails
        df = pass_fail_no_d(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Intermediate2']
years = ['2003', '2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Rename comp column
        df = df.rename(columns={'COMP': 'D'})
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Higher']
years = ['2003']

# Define sheetnames to grab and header to search for
sheets = ['4a']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Rename comp column
        df = df.rename(columns={'COMP': 'D'})
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level

        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Higher']
years = ['2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheets = ['3a']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Rename comp column
        df = df.rename(columns={'COMP': 'D'})
        
        # Calculate pass and fails
        df = pass_fail(df)
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level

        # Append to df list
        frames.append(dfl)

In [ ]:
real_level = 'Old_Higher'
level = 'Standard_Grade' # Stats hidden away in SG document
years = ['2001', '2000']

sheet = 'hg3a'
header = 'subject'

for year in years:
    
    # Read and tidy data
    df = read_single_sheet(year, level, header, sheet)
    df = tidy_df(df)

    # Calculate pass and fails
    df = pass_fail(df)

    # Change into long format
    dfl = pd.melt(df, id_vars =['SUBJECT'], 
            value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                    var_name ='GRADE', value_name ='COUNT')

    # Add in year and level columns
    dfl['YEAR'] = int(year)
    dfl['LEVEL'] = real_level
    
    # Append to df list
    frames.append(dfl)

In [ ]:
# Define level and years
level = 'National_4'
years = ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']

# Define sheetnames to grab and header to search for
sheet1 = 'table 1'
sheet2 = 'table 2'
header = 'TITLE'

# Create empty dataframe
dfl = pd.DataFrame()

for year in years:
        
    ### read in first sheet

    # Read and tidy data
    df1 = read_single_sheet(year, level, header, sheet1)
    df1 = tidy_df(df1)

    # Select correct columns
    df1 = df1[['SUBJECT', year]]

    # Sort headers
    df1.columns = ['SUBJECT', 'COUNT']

    # Add in year and level columns
    df1['YEAR'] = int(year)
    df1['LEVEL'] = level
    df1['GRADE'] = 'ENTRIES'

    # Append to df list
    frames.append(df1)

    ### read in second sheet

    # Read and tidy data
    df2 = read_single_sheet(year, level, header, sheet2)
    df2 = tidy_df(df2)

    # Select correct columns
    df2 = df2[['SUBJECT', year]]

    # Sort headers
    df2.columns = ['SUBJECT', 'COUNT']

    # Add in year and level columns
    df2['YEAR'] = int(year)
    df2['LEVEL'] = level
    df2['GRADE'] = 'PASS'

    # Append to df list
    frames.append(df2)

    ### Calculate NAs

    # Copy first orginal df
    df3 = df1.copy()

    # Change grade to b - represents NAs
    df3['GRADE'] = 'NA'

    # Change count to difference between entries and passes
    df3['COUNT'] = df1['COUNT'] - df2['COUNT']

    # Append to df list
    frames.append(df3)

In [ ]:
# Define level and years
levels = ['Standard_Grade']
years = ['2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', 
                 '2005', '2004', '2003']

# Define sheetnames to grab and header to search for
sheets = ['sg3']
header = 'title|subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        # Read and tidy data
        df = read_sheet(year, level, header, sheets)
        df = tidy_df(df)
        
        # Replace nans with zeros
        #df = df.fillna(0)
        
        # Create new NA (no award) column with value from grade 7 column
        df['NA'] = df['7']
        
        # Add column with pass count
        df['PASS'] = df['ENTRIES'] - df['NA']
        
        # Change into long format
        dfl = pd.melt(df, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', '1', '2', '3', '4', '5', '6', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)

In [ ]:
# Define level and years
levels = ['Standard_Grade']
years = ['2002', '2001', '2000']

# Define sheetnames to grab and header to search for
sheet1 = 'sg4a'
sheet2 = 'sg4b'
header = 'subject'

# Create empty dataframe
dfl = pd.DataFrame()

for level in levels:
    
    for year in years:
        
        ### read in first sheet (males)
        
        # Read and tidy data
        df1 = read_single_sheet(year, level, header, sheet1)
        df1 = tidy_df(df1)
        
        # Create new NA (no award) column with value from grade 7 column
        df1['NA'] = df1['7']
        
        # Add column with pass count
        df1['PASS'] = df1['ENTRIES'] - df1['NA']
        
        ### read in second sheet (females)
        
        # Read and tidy data
        df2 = read_single_sheet(year, level, header, sheet2)
        df2 = tidy_df(df2)
        
        # Create new NA (no award) column with value from grade 7 column
        df2['NA'] = df2['7']
        
        # Add column with pass count
        df2['PASS'] = df2['ENTRIES'] - df1['NA']
        
        ### Add together values
        df3 = pd.DataFrame()
        
        df3['SUBJECT'] = df1['SUBJECT']
        
        df3['ENTRIES'] =  df1['ENTRIES'] + df2['ENTRIES']
        df3['PASS'] =  df1['PASS'] + df2['PASS']
        df3['NA'] =  df1['NA'] + df2['NA']
        
        df3['1'] =  df1['1'] + df2['1']
        df3['2'] =  df1['2'] + df2['2']
        df3['3'] =  df1['3'] + df2['3']
        df3['4'] =  df1['4'] + df2['4']
        df3['5'] =  df1['5'] + df2['5']
        df3['6'] =  df1['6'] + df2['6']
        
        # Change into long format
        dfl = pd.melt(df3, id_vars =['SUBJECT'], 
                value_vars = ['ENTRIES', '1', '2', '3', '4', '5', '6', 'PASS', 'NA'],
                        var_name ='GRADE', value_name ='COUNT')
        
        
        # Add in year and level columns
        dfl['YEAR'] = int(year)
        dfl['LEVEL'] = level
        
        # Append to df list
        frames.append(dfl)
        
#         # Rough conversion to N5 grades idea - 1,2 > a, 3,4 > b, 5 > c, 6 > d, 7 > na
#         dfs['A'] = dfs['1'] + dfs['2']
#         dfs['B'] = dfs['3'] + dfs['4']
#         dfs['C'] = dfs['5']
#         dfs['D'] = dfs['6']

In [ ]:
years = ['1999', '1998', '1997', '1996', '1995', '1994', '1993',
                    '1992', '1991', '1990', '1989', '1988', '1987', '1986']

level = 'Standard_Grade'

# Define sheetnames to grab and header to search for
sheet = 'std grade'
header = 'subject'

for year in years:
    
    # Read and tidy data
    df = read_old_single_sheet(year, header, sheet, 2)
    df = tidy_df(df)
    df = select_old_columns(df)
    
    # Replace nans with zeros
    df = df.fillna(0)

    # Create new NA (no award) column with value from grade 7 column
    df['NA'] = df['7']

    # Add column with pass count
    df['PASS'] = df['ENTRIES'] - df['NA']

    # Change into long format
    dfl = pd.melt(df, id_vars =['SUBJECT'], 
            value_vars = ['ENTRIES', '1', '2', '3', '4', '5', '6', 'PASS', 'NA'],
                    var_name ='GRADE', value_name ='COUNT')


    # Add in year and level columns
    dfl['YEAR'] = int(year)
    dfl['LEVEL'] = level
    
    # Append to df list
    frames.append(dfl)

In [ ]:
years = ['1999', '1998', '1997', '1996', '1995', '1994', '1993',
                    '1992', '1991', '1990', '1989', '1988', '1987', '1986']

level = 'Higher'

# Define sheetnames to grab and header to search for
sheet = 'higher'
header = 'subject'

for year in years:
    
    # Read and tidy data
    df = read_old_higher_single_sheet(year, header, sheet)
    df = tidy_df(df)
    df = select_old_higher_columns(df)
    
    # Replace nans with zeros
    df = df.fillna(0)

    # Calculate pass and fails
    df = pass_fail(df)
    
    # Change into long format
    dfl = pd.melt(df, id_vars =['SUBJECT'], 
            value_vars = ['ENTRIES', 'A', 'B', 'C', 'D', 'PASS', 'NA'],
                    var_name ='GRADE', value_name ='COUNT')

    # Add in year and level columns
    dfl['YEAR'] = int(year)
    dfl['LEVEL'] = level
    
    # Append to df list
    frames.append(dfl)

In [ ]:
years = ['1988', '1987', '1986']

level = 'Ordinary_Grade'

# Define sheetnames to grab and header to search for
sheet = 'ordinary grade'
header = 'subject'

for year in years:
    
    # Read and tidy data
    df = read_old_single_sheet(year, header, sheet, 3)
    df = tidy_df(df)
    df = select_old_ordinary_columns(df)
    
    # Replace nans with zeros
    df = df.fillna(0)
    
    df = pass_fail_ordinary(df)

    # Change into long format
    dfl = pd.melt(df, id_vars =['SUBJECT'], 
            value_vars = ['ENTRIES', '1', '2', '3', '4', '5', 'PASS', 'NA'],
                    var_name ='GRADE', value_name ='COUNT')


    # Add in year and level columns
    dfl['YEAR'] = int(year)
    dfl['LEVEL'] = level
    
    # Append to df list
    frames.append(dfl)

# Tidy up

In [ ]:
# Concat list of dfs together
dft = pd.concat(frames)

# Drop row with dodgy Subject values
dft = dft[dft.SUBJECT != '- AS PERCENTAGES']
dft = dft[dft.SUBJECT != 'AS PERCENTAGES']
dft = dft[dft.SUBJECT != 'AS PERCENTAGE']
dft = dft[dft.SUBJECT != 'SUBTOTALS']
dft = dft[dft.SUBJECT != 'SUBTOTAL']

# Sort categories name changes
dft['SUBJECT'] = dft['SUBJECT'].str.replace('TOTALS', 'TOTAL')

# Drop rows with any nans
dft = dft.dropna()

# Dictionary to map level replacements
levels = {'Advanced_Higher': 'ADVANCED HIGHER',
         'Higher': 'HIGHER',
         'Intermediate1': 'INTERMEDIATE 1',
         'Intermediate2': 'INTERMEDIATE 2',
         'National_4': 'NATIONAL 4',
         'National_5': 'NATIONAL 5',
         'New_Higher': 'NEW HIGHER',
         'Old_Higher': 'OLD HIGHER',
         'Ordinary_Grade': 'ORDINARY GRADE',
         'Standard_Grade': 'STANDARD GRADE'}

# Replace level values using dictionary
dft = dft.replace({'LEVEL': levels})

# Dictionary to map subject replacements
subs = {'GÁIDHLIG': 'GÀIDHLIG',
        'GAIDHLIG': 'GÀIDHLIG',
        'GAELIC NATIVE SPEAKERS': 'GÀIDHLIG'}

# Replace level values using dictionary
dft = dft.replace({'SUBJECT': subs})


## Remove writing parts of language courses

# Create list of unique subjects
subls = sorted(dft['SUBJECT'].unique())
# Look for items with tag in the name
matches = [match for match in subls if ' WRITING' in match]
# Remove matches
dft = dft.loc[~dft['SUBJECT'].isin(matches)]

## Remove optional paper III of language courses

# Create list of unique subjects
subls = sorted(dft['SUBJECT'].unique())
# Look for items with tag in the name
matches = [match for match in subls if ' OPTIONAL PAPER III' in match]
# Remove matches
dft = dft.loc[~dft['SUBJECT'].isin(matches)]

# Sort out row order and reset index
dft = dft.sort_values(['YEAR', 'LEVEL', 'SUBJECT', 'GRADE'])
dft = dft.reset_index(drop=True)

In [ ]:
# Select a subset of the df
#dft.loc[(dft['SUBJECT'] == 'HEALTH & SOCIAL CARE')]
#sorted(dft['LEVEL'].unique())
dft.head()

# Tidy up some more

# Export to .csv

In [ ]:
dft.to_csv('./output_csvs/1986_2023_data.csv')

In [ ]:
#dfu.to_csv('./output_csvs/1986_2023_data_simplified.csv')